In [1]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
# from sklearnex import patch_sklearn
# patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


max_length=128

c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
PATH='train_file.json'
BATCH_SIZE=16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "roberta-base"
label_encoder = LabelEncoder()

In [3]:
# read json
with open(PATH) as f:
    data = json.load(f)
    # to pandas
    df = pd.DataFrame(data)


In [4]:
df.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you you had sex with Ursula?!, Uh, a litt...","[1.0, 1.0, 0.0, 0.0, 0.0]"
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]"
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0.0, 0.0, 1.0]"
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0.0, 0.0, 1.0]"


In [5]:
sentence_len = []
text=[]
EOS='</s>'
SEP='[SEP]'
SOS='<s>'

for i in range(len(df)):
    uterances=df.iloc[i]['utterances']
    speaker=df.iloc[i]['speakers']
    length=0
    sentence=SOS+' '
    for (speaker,uterance) in zip(speaker,uterances):
        sentence+=speaker+':'+uterance+' '+SEP+' '

        length+=len(uterance.split())
    if(length==2):
        print(sentence)
        print(df.iloc[i])
    sentence_len.append(length)
    text.append(sentence+EOS)
    # print(length)
    # break

<s> Phoebe:No. [SEP] Phoebe:No! [SEP] 
episode          utterance_915
speakers      [Phoebe, Phoebe]
emotions      [neutral, anger]
utterances          [No., No!]
triggers            [0.0, 0.0]
Name: 968, dtype: object
<s> Phoebe:No. [SEP] Phoebe:No! [SEP] 
episode          utterance_915
speakers      [Phoebe, Phoebe]
emotions      [neutral, anger]
utterances          [No., No!]
triggers            [0.0, 0.0]
Name: 3984, dtype: object


In [6]:
# config =AutoConfig.from_pretrained(model_id)
# config.update({"id2label": label_to_id})
# tokenizer = RobertaTokenizerFast.from_pretrained(model_id)
# model = RobertaForSequenceClassification.from_pretrained(model_id,config=config)

In [7]:
text_df=pd.DataFrame(text,columns=['text'])
print(len(text),len(set(text)))
text_df.head()

6740 4179


,text
0,<s> Phoebe:You-you you had sex with Ursula?! [...
1,"<s> Monica:Dad, please don't pick your teeth o..."
2,"<s> Older Scientist:Dr. Geller, there's a seat..."
3,"<s> Monica:So, how'd the lasagne go over? [SEP..."
4,<s> Kate:Become a drama critic! [SEP] The Dire...


In [8]:
label_encoding ={
    'x':0,
    'surprise':1,
    'sadness':2,
    'anger':3,
    'fear':4,
    'disgust':5,
    'joy':6,
    'neutral':7
}


In [9]:
y_train=[]
for i in df["emotions"]:
    lst=[]
    for j in i:
        lst.append(label_encoding[j])

    lst.extend([0]*(max_length-len(lst)))
    y_train.append(lst)
x_train=text_df['text']

In [10]:
y_train[0],y_train[1]

([1,
  4,
  1,
  2,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [5,
  5,
  3,
  2,
  1,
  3,
  7,
  3,
  3,
  3,
  3,
  4,
  7,
  6,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [38]:
from torch import nn
config = AutoConfig.from_pretrained(model_id)
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)
config.update({"id2label": label_encoding})
roberta_model = RobertaForSequenceClassification.from_pretrained(model_id,config=config)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
# x_train_tokenized = tokenizer(text_df["text"].tolist(),padding=True,truncation=True,max_length=128)
# x_train_tokenized[0]

In [40]:
# model = model.to(device)
# model

In [41]:
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self,encodings,labels=None):
#         self.labels = labels
#         self.encodings = encodings
    
#     def __getitem__(self,idx):
#         item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
#         if self.labels is not None:
#             item["labels"]  = torch.tensor(self.labels[idx])
#         return item
# #             return input_data["input_ids"],input_data["attention_mask"],self.labels[idx]


#     def __len__(self):
#         return len(self.encodings["input_ids"])

In [42]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer,labels):
        self.data=data
        self.tokenizer = tokenizer
        self.labels = labels
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):   
        encoded_pair = self.tokenizer(self.data[idx],max_length=128,truncation=True,return_tensors="pt",padding="max_length")
        # print(encoded_pair)
        input_ids = encoded_pair['input_ids'].squeeze(0)
        attention_mask = encoded_pair['attention_mask'].squeeze(0)
        # token_type_ids = encoded_pair['token_type_ids'].squeeze(0)
        return input_ids,attention_mask,torch.tensor(self.labels[idx])

In [43]:
train_dataset = Dataset(x_train,tokenizer,y_train)

In [44]:
for i in train_dataset:
    for j in i:
        print(len(j))
    break

128
128
128


In [45]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=False)

Roberta output should be of size BATCH_SIZExlabels, we need linear layer to output BATCH_SIZExlabels as emotions in one way or another after we will do softmax. Now manage this. 

In [49]:

class EmotionClassifier(nn.Module):
    def __init__(self, roberta_model, num_labels):
        super(EmotionClassifier, self).__init__()
        self.roberta = roberta_model
        self.linear = nn.Linear(BATCH_SIZE, num_labels) # edit this to match out desirec size
        self.Softmax = nn.Softmax() 

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
        print(outputs.logits.shape) # now working correctly
        pooled_output = outputs['logits']  
        logits = self.linear(pooled_output)
        probabilities = self.Softmax(logits)
        return probabilities

In [50]:
num_labels = 8
model = EmotionClassifier(roberta_model, num_labels)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=0.005)
criterion = nn.CrossEntropyLoss()

from tqdm import tqdm
import gc
train_losses=[]
val_losses=[]
def train_epoch(model, optimizer,epoch):
    model.train()
    losses = 0
    progress = tqdm(train_dataloader, desc=f"Epoch:{epoch}",total=len(train_dataloader), leave=False)
    i=0
    for batch in progress:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        # token_type_ids = batch[2].to(device)
        labels = batch[2].to(device)
        optimizer.zero_grad()
        probabilities = model(input_ids, attention_mask=attention_mask)
        loss = criterion(probabilities, labels.float())
        losses += loss.item()
        loss.backward()
        optimizer.step()
        del input_ids
        del attention_mask
        del labels
        del logits
        gc.collect()
        torch.cuda.empty_cache()
        progress.set_postfix({'training_loss': f'{loss.item()/len(batch):.3f}'})
        break
    x = losses / len(list(train_dataloader))
    train_losses.append(x)
#     wandb.log({'epoch':epoch,'train_loss':x})
    tqdm.write(f"Epoch:{epoch}, Avg Train Loss: {x}")
    gc.collect()
    torch.cuda.empty_cache()
    return x



def evaluate(model,val_dataloader):
    with torch.no_grad():
        model.eval()
        losses = 0
        
        total_logits=[]
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            logits=model(input_ids,attention_mask=attention_mask).logits.to(torch.float64).view(-1)*5
            total_logits.extend(list(logits.cpu().detach().numpy()))
            
        return total_logits

In [51]:
gc.collect()
torch.cuda.empty_cache()
for epoch in range(1, 2):
    train_loss = train_epoch(model, optimizer,epoch)
    val_loss = evaluate(model)
    torch.save(model.state_dict(), f"model1A_epoch{epoch}.pth")
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}"))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x8 and 16x8)